In [1]:
import math, os, sys
import json
import ast
import plotly.express as px
import pandas as pd
from datetime import datetime
sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [ ]:
from neuroglancer.models import UrlModel, LayerData, CenterOfMass, Structure
from brain.models import Animal

In [ ]:
id = 246
row_dict = {}
rows = CenterOfMass.objects.filter(prep__prep_id=prep_id).filter(structure=1) \
.filter(active=True).order_by('structure', 'updated')
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section, row.updated, row.person_id]
    print(f'{structure}\t {row.x}\t {row.y}\t{row.section}\t{row.updated}\t {row.person_id}')
row_dict['10N_L']

In [ ]:
layer_keys = []
index = 1
urlModels = UrlModel.objects.all()
for urlModel in urlModels:
    json_txt = json.loads(urlModel.url)
    if 'layers' in json_txt:
        layers = json_txt['layers']
        for layer in layers:
            if 'annotations' in layer and 'name' in layer:
                annotation = layer['annotations']
                layer_name = layer['name']
                if len(annotation) > 0:
                    print(index, urlModel.comments, urlModel.id, layer_name, len(annotation))
                    index += 1
                    layer_keys.append({"id":urlModel.id, 
                             "description":urlModel.comments, 
                             "layer_name":layer_name})
                

In [ ]:
for layer in layer_keys:
    print(layer)

In [ ]:
id = 246
urlModel = UrlModel.objects.get(pk=id)
points = []
layer_name = 'COM'
json_txt = json.loads(urlModel.url)
layers = {}
if 'layers' in json_txt:
    layers = json_txt['layers']
    for layer in layers:
        if 'annotations' in layer:
            annotation = layer['annotations']
            if len(annotation) > 0 and layer_name in layer['name']:
                #d = [row['point'] for row in annotation]
                print(layer_name, len(annotation))
                #print(json.dumps(annotation, indent=2))
                points = annotation

In [ ]:
points

In [ ]:
layers = {}
json_txt = json.loads(urlModel.url)
if 'layers' in json_txt:
    layers = json_txt['layers']
    for layer in layers:
        if 'annotations' in layer:
            lname = layer['name']
            if 'com' in lname.lower():
                annotation = layer['annotations']
                for com in annotation:
                    x = com['point'][0]
                    y = com['point'][1]
                    z = com['point'][2]
                    if 'description' in com:
                        abbreviation = com['description']

                        structure = Structure.objects.get(abbreviation=abbreviation)
                        prep = Animal.objects.get(prep_id=urlModel.animal)
                        centerOfMass = CenterOfMass(prep=prep, structure=structure,x=x,y=y,section=z,active=True,created=datetime.now())
                        print(centerOfMass.prep.prep_id, abbreviation,structure.abbreviation,x,y,z,centerOfMass.created)
                        #centerOfMass.save()


In [ ]:
reference_centers

In [ ]:
R, t = align_atlas(reference_centers, reference_scales=(0.325, 0.325, 20))

In [ ]:
df = pd.DataFrame(reference_centers.values(), columns=['X', 'Y', 'Section'])
df.head()
fig = px.scatter_3d(df, x='X', y='Y', z='Section',
              color='Section', opacity=0.7)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
#print(R)
#print(t)
rl = R. tolist()
tl = t.tolist()
data = {}
data['transformation'] = rl
data['translation'] = tl
print(rl_json_str)
print()
print(tl_json_str)